#Imports

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Montando tabela patients_icustays

###Começaremo pela tabela Pacients pois
Com essa tabela conseguiremos:

* Idade
* genero


In [ ]:
# Lendo a Tabela
patients = pd.read_csv('/content/drive/MyDrive/patients.csv.gz')

In [ ]:
# Mostrando a tabela pacientes
patients.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,NaN


###Agora que ja temos a tabela patients podemos ir para a tabela icustays para conseguir:

* hadm_id
* stay_id
* apenas pacientes da UTI


---


**essa tabela é importante pois so tem pacientes de UTI e com ela podemos pegar a primeira internação de cada paciente que é importante para não haver interferência humana**

In [ ]:
# Lendo a tabela
icustays = pd.read_csv('/content/drive/MyDrive/icustays.csv.gz')

In [ ]:
# Mostrando a tabela icustays
icustays.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
2,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
3,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113
4,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588


#### Agora iremos filtrar somente a primeira estadia de cada paciente

In [ ]:
# Ordena o DataFrame por paciente e por data de entrada
icustays_sorted = icustays.sort_values(['subject_id', 'intime'])

# Para cada paciente (subject_id), pega a primeira linha (primeira estadia)
primeira_estadia = icustays_sorted.groupby('subject_id').first().reset_index()

# primeira_estadia agora tem só a primeira estadia de cada paciente
primeira_estadia.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
2,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
3,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588
4,10001884,26184834,37510196,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817


###Agora iremos fazer a junção entre as bases icustays e patients com apenas as colunas que queremos
###( 'subject_id', 'hadm_id', 'stay_id', 'gender', 'anchor_age' )
###para obtermos nossa base patients_icustays

In [ ]:
# Fazendo a junção das tabelas pelo subject_id
df_final = primeira_estadia.merge(
    patients[['subject_id', 'gender', 'anchor_age']],  # seleciona só as colunas necessárias
    on='subject_id',
    how='left'  # ou 'inner' se quiser só pacientes que aparecem em ambos
)

# Selecionando e organizando as colunas na ordem que pediu
df_final = df_final[['subject_id', 'hadm_id', 'stay_id', 'gender', 'anchor_age']]
#mostrando a base final
df_final.head()

,subject_id,hadm_id,stay_id,gender,anchor_age
0,10000032,29079034,39553978,F,52
1,10000980,26913865,39765666,F,73
2,10001217,24597018,37067082,F,55
3,10001725,25563031,31205490,F,46
4,10001884,26184834,37510196,F,68


# Montando a tabela chartevents como queremos

Após estudar a documentação da MIMIC IV 2.2 entedemos que para encontrar os sinais vitais temos que ir até a tabela chartevents então vamos usa-la para conseguir os dados dos sinais vitais dos pacientes e horario que foram tirados.

### lendo o chartevents para começar a modifica-lo

In [ ]:
# IDs desejados
itemids = [223762,223761, 220045, 220210, 220179, 220180, 220181, 220277,226730,226512]

# Lendo em chunks para não estourar a memória
chunks = pd.read_csv('/content/drive/MyDrive/chartevents.csv.gz', chunksize=500000)

# Filtrando os chunks e concatenando os resultados
df_filtrado = pd.concat(
    chunk[chunk['itemid'].isin(itemids)]
    for chunk in chunks
)

# Salvando em formato Parquet
df_filtrado.to_parquet('chartevents.parquet')

Agora que podemos ver como é realmente a tabela chartevents podemos perceber que:

* temos as colunas subject_id, stay_id e hadm_id para podermos fazer uma junção com a nossa outra base
* temos a hora que foi tirado o sinal vital em charttime
* temos 1 itemid para cada sinal vital
* temos o valor de cada sinal vital e sua respectiva unidade de medida


---

Mas como iremos saber o itemid de cada sinal vital ? lendo a documentação chegamos na tabela d_itens

In [ ]:
df_filtrado = pd.read_parquet('chartevents.parquet')
df_filtrado.head()

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10000032,29079034,39553978,47007.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220179,82,82.0,mmHg,0.0
1,10000032,29079034,39553978,47007.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220180,59,59.0,mmHg,0.0
2,10000032,29079034,39553978,47007.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220181,63,63.0,mmHg,0.0
3,10000032,29079034,39553978,47007.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220045,94,94.0,bpm,0.0
4,10000032,29079034,39553978,47007.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220179,85,85.0,mmHg,0.0


### lendo a tabela itens para achar os id's necessarios para a nossa tabela final

In [ ]:
#carregando a tabela
itens = pd.read_csv('d_items.csv.gz')

In [ ]:
#mostrando a tabela
itens.head()

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220001,Problem List,Problem List,chartevents,General,NaN,Text,NaN,NaN
1,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,NaN,Date and time,NaN,NaN
2,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
3,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN
4,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN


Como podemos ver na tabela temos o itemid com seu respectivo rotulo.
Então selecionando os rotulos que nos precisavamos de maneira braçal. Chegamos nesses valores de itemid

In [ ]:
df_filtrado = itens[itens['itemid'].isin([223761, 220045, 220210, 220179, 220180, 220181, 220277,226512, 226730])]

Nessa tabela podemos ver os rotulos que precisamos para a tabela final seu respectivo itemid e a unidade de medida

In [ ]:
df_filtrado

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
2,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
24,220179,Non Invasive Blood Pressure systolic,NBPs,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
25,220180,Non Invasive Blood Pressure diastolic,NBPd,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
26,220181,Non Invasive Blood Pressure mean,NBPm,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
28,220210,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN
36,220277,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN
337,223761,Temperature Fahrenheit,Temperature F,chartevents,Routine Vital Signs,°F,Numeric,NaN,NaN
1856,226512,Admission Weight (Kg),Admission Weight (Kg),chartevents,General,kg,Numeric,NaN,NaN
1945,226730,Height (cm),Height (cm),chartevents,General,cm,Numeric,NaN,NaN


Agora que sabemos os itemid que precisamos podemos voltar para a tabela chartevents onde iremos filtrar as colunas desejadas e filtrar, tambem, a coluna itemid apenas pelos que precisamos.

### Filtrando pelas colunas desejadas

In [ ]:
# esse codigo filtra as colunas desejadas
colunas_desejadas = ['subject_id', 'hadm_id', 'stay_id', 'charttime','itemid','valuenum',]
df_filtrado = df_filtrado[colunas_desejadas]

In [ ]:
#Mostando a tabela chartevents filtrada
df_filtrado.head()

,subject_id,hadm_id,stay_id,charttime,itemid,valuenum
0,10000032,29079034,39553978,2180-07-23 21:01:00,220179,82.0
1,10000032,29079034,39553978,2180-07-23 21:01:00,220180,59.0
2,10000032,29079034,39553978,2180-07-23 21:01:00,220181,63.0
3,10000032,29079034,39553978,2180-07-23 22:00:00,220045,94.0
4,10000032,29079034,39553978,2180-07-23 22:00:00,220179,85.0


Agora precisamos remover os outliers da nossa base

para isso tivemos que fazer uma pesquisa para identificar até onde o sinal vital poderia ir e claro deixamos um limite a mais.


### Removendo possiveis outliers

In [ ]:
limites_clinicos = {
    223762: (30.0, 43.0),     # Temperatura (°C)
    223761: (86.0, 109.4),    # Temperatura (°F)
    220210: (5, 60),          # FR
    220045: (30, 220),        # FC
    220179: (50, 250),        # PAS
    220180: (30, 150),        # PAD
    220181: (40, 170),        # PAM
    220277: (50, 100),        # SpO2
    226730: (120, 220),       # altura
    226512: (30, 250)         # peso
}

# Aplica os filtros
filtros = []
for itemid, (min_val, max_val) in limites_clinicos.items():
    filtro = (df_filtrado['itemid'] == itemid) & (df_filtrado['valuenum'] >= min_val) & (df_filtrado['valuenum'] <= max_val)
    filtros.append(filtro)

# Combina todos os filtros com OR
df_filtrado = df_filtrado[pd.concat(filtros, axis=1).any(axis=1)]

### confirmando que os valores estão dentro dos intervalo sugerido

In [ ]:
# Filtra apenas os itemid presentes nos limites_clinicos
df_filtrados = df_filtrado[df_filtrado['itemid'].isin(limites_clinicos.keys())]

# Verificação de valores fora dos limites
valores_invalidos = []

for itemid, (min_val, max_val) in limites_clinicos.items():
    df_item = df_filtrados[df_filtrados['itemid'] == itemid]
    fora_do_intervalo = df_item[(df_item['valuenum'] < min_val) | (df_item['valuenum'] > max_val)]
    if not fora_do_intervalo.empty:
        valores_invalidos.append(fora_do_intervalo)

# Junta todos os dados inválidos (se houver)
if valores_invalidos:
    df_fora_intervalos = pd.concat(valores_invalidos)
    print(" Existem valores fora dos limites clínicos definidos:")
    display(df_fora_intervalos)
else:
    print(" Todos os valores estão dentro dos limites clínicos.")


 Todos os valores estão dentro dos limites clínicos.


Agora que temos a chartevents sem outliers. Iremos junta-la com a icustays_patients.

## Juntando o chartevents limpo com a tabela icustays_patients

In [ ]:
# Codigo para fazer a junção das tabelas

df_merged = df_filtrado.merge(
    df_final,
    on=['subject_id', 'hadm_id', 'stay_id'],
    how='inner'
)

In [ ]:
#mostrando a base fruto da junção entre chartevents e icustays_patients
df_merged.head()

,subject_id,hadm_id,stay_id,charttime,itemid,valuenum,gender,anchor_age
0,10000032,29079034,39553978,2180-07-23 21:01:00,220179,82.0,F,52
1,10000032,29079034,39553978,2180-07-23 21:01:00,220180,59.0,F,52
2,10000032,29079034,39553978,2180-07-23 21:01:00,220181,63.0,F,52
3,10000032,29079034,39553978,2180-07-23 22:00:00,220045,94.0,F,52
4,10000032,29079034,39553978,2180-07-23 22:00:00,220179,85.0,F,52


Nessa base agora temos a mais:
* sinais vitais em itemid
* hora que o sinal vital foi tirado em chartevents
* valor do sinal vital em valuenum


---

Agora teremos que fazer cada itemid virar uma coluna para isso usaremos uma tecnica chamado pivotamento que faremos logo abaixo

## fazer o pivotamento da tabela

In [ ]:
# codigo que fará o pivotamento da tabela
df_pivotado = df_merged.pivot_table(
    index=['subject_id', 'hadm_id', 'stay_id', 'charttime', 'gender', 'anchor_age'],
    columns='itemid',
    values='valuenum'
).reset_index()

In [ ]:
# mostrando a tabela
df_pivotado.head()

itemid,subject_id,hadm_id,stay_id,charttime,gender,anchor_age,FC,PAS,PAD,PAM,FR,SpO2,Temperatura (°F),Temperatura (°C),Peso,Altura
0,10000032,29079034,39553978,2180-07-23 12:36:00,F,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.4,152.0
1,10000032,29079034,39553978,2180-07-23 14:00:00,F,52,NaN,NaN,NaN,NaN,NaN,NaN,98.7,NaN,NaN,NaN
2,10000032,29079034,39553978,2180-07-23 14:11:00,F,52,NaN,84.0,48.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN
3,10000032,29079034,39553978,2180-07-23 14:12:00,F,52,91.0,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN
4,10000032,29079034,39553978,2180-07-23 14:13:00,F,52,NaN,NaN,NaN,NaN,NaN,98.0,NaN,NaN,NaN,NaN


In [ ]:
# Renomeando colunas
nomes_clinicos = {
    223762: 'Temperatura (°C)',
    223761: 'Temperatura (°F)',
    220210: 'FR',
    220045: 'FC',
    220179: 'PAS',
    220180: 'PAD',
    220181: 'PAM',
    220277: 'SpO2',
    226730: 'Altura',
    226512: 'Peso'
}

df_pivotado.rename(columns=nomes_clinicos, inplace=True)

In [ ]:
#Tabela após renomear as colunas
df_pivotado.head()

itemid,subject_id,hadm_id,stay_id,charttime,gender,anchor_age,FC,PAS,PAD,PAM,FR,SpO2,Temperatura (°F),Temperatura (°C),Peso,Altura
0,10000032,29079034,39553978,2180-07-23 12:36:00,F,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.4,152.0
1,10000032,29079034,39553978,2180-07-23 14:00:00,F,52,NaN,NaN,NaN,NaN,NaN,NaN,98.7,NaN,NaN,NaN
2,10000032,29079034,39553978,2180-07-23 14:11:00,F,52,NaN,84.0,48.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN
3,10000032,29079034,39553978,2180-07-23 14:12:00,F,52,91.0,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN
4,10000032,29079034,39553978,2180-07-23 14:13:00,F,52,NaN,NaN,NaN,NaN,NaN,98.0,NaN,NaN,NaN,NaN


#Salvando nosso dataframe pivotado

In [ ]:
# Aqui estou apenas salvando o dataframe para não ter que fazer tudo novamente
df_pivotado.to_csv('/content/drive/MyDrive/df_pivotado.csv', index=False)
df_pivotado.to_parquet('/content/drive/MyDrive/df_pivotado.parquet', index=False)

NameError: name 'df_pivotado' is not defined

#Criando as janelas para o LSTM

In [ ]:
df = df_pivotado

In [ ]:
# apenas lendo nosso dataframe
df = pd.read_parquet('/content/drive/MyDrive/df_pivotado.parquet')

In [ ]:
# mostrando como estamos até agora
df.head()

itemid,subject_id,hadm_id,stay_id,charttime,gender,anchor_age,FC,PAS,PAD,PAM,FR,SpO2,Temperatura (°F),Temperatura (°C),Peso,Altura
0,10000032,29079034,39553978,2180-07-23 12:36:00,F,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.4,152.0
1,10000032,29079034,39553978,2180-07-23 14:00:00,F,52,NaN,NaN,NaN,NaN,NaN,NaN,98.7,NaN,NaN,NaN
2,10000032,29079034,39553978,2180-07-23 14:11:00,F,52,NaN,84.0,48.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN
3,10000032,29079034,39553978,2180-07-23 14:12:00,F,52,91.0,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN
4,10000032,29079034,39553978,2180-07-23 14:13:00,F,52,NaN,NaN,NaN,NaN,NaN,98.0,NaN,NaN,NaN,NaN


##Aqui iremos criar as janelas de 4 horas se quiser mudar a quantidade altere nesse codigo

* Com essas janelas teremos uma quantidade de dados a cada 4 horas agrupados
* Como nessa vez estou usando 20 janelas teremos as primeiras 20*4 horas de cada paciente
* A janela começa a partir do primeiro charttime de cada paciente
* Caso o paciente não tenha dados para 20 janelas não será criado dados para isso ficará com o máximos de janelas que ele consegue

In [ ]:
df = mimic

In [ ]:
# Converte para datetime, se necessário
df['charttime'] = pd.to_datetime(df['charttime'])

# Ordena para garantir consistência
df = df.sort_values(['subject_id', 'stay_id', 'charttime']).reset_index(drop=True)

# Função para criar no máximo 10 janelas de 4h
def atribuir_janelas_10x4h(grupo):
    inicio = grupo['charttime'].min()

    # Diferença em horas desde o início
    delta_horas = (grupo['charttime'] - inicio).dt.total_seconds() / 3600.0

    # Define índice da janela
    janela_index = np.floor(delta_horas / 4).astype(int)

    # Só mantém as janelas até a 9 (total 10 janelas)
    mask = janela_index < 20
    grupo = grupo.loc[mask].copy()

    grupo['janela_index'] = janela_index[mask].values
    grupo['inicio_janela'] = inicio + pd.to_timedelta(grupo['janela_index'] * 4, unit='h')

    return grupo

# Aplica por paciente/internação
df_janelado = df.groupby(['subject_id', 'stay_id'], group_keys=False).apply(atribuir_janelas_10x4h)

/tmp/ipython-input-3925007289.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_janelado = df.groupby(['subject_id', 'stay_id'], group_keys=False).apply(atribuir_janelas_10x4h)


In [ ]:
df_janelado

itemid,subject_id,hadm_id,stay_id,charttime,gender,anchor_age,FC,PAS,PAD,PAM,FR,SpO2,Temperatura (°F),Temperatura (°C),Peso,Altura,sepse,janela_index,inicio_janela
0,10000032,29079034,39553978,2180-07-23 12:36:00,F,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.4,152.0,0,0,2180-07-23 12:36:00
1,10000032,29079034,39553978,2180-07-23 14:00:00,F,52,NaN,NaN,NaN,NaN,NaN,NaN,98.7,NaN,NaN,NaN,0,0,2180-07-23 12:36:00
2,10000032,29079034,39553978,2180-07-23 14:11:00,F,52,NaN,84.0,48.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2180-07-23 12:36:00
3,10000032,29079034,39553978,2180-07-23 14:12:00,F,52,91.0,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,0,0,2180-07-23 12:36:00
4,10000032,29079034,39553978,2180-07-23 14:13:00,F,52,NaN,NaN,NaN,NaN,NaN,98.0,NaN,NaN,NaN,NaN,0,0,2180-07-23 12:36:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6060807,19999987,23865745,36195440,2145-11-04 19:31:00,F,57,NaN,163.0,120.0,131.0,NaN,NaN,NaN,NaN,NaN,NaN,0,11,2145-11-04 17:39:00
6060808,19999987,23865745,36195440,2145-11-04 19:37:00,F,57,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,0,11,2145-11-04 17:39:00
6060809,19999987,23865745,36195440,2145-11-04 20:00:00,F,57,92.0,NaN,NaN,NaN,23.0,100.0,100.1,NaN,NaN,NaN,0,11,2145-11-04 17:39:00
6060810,19999987,23865745,36195440,2145-11-04 21:00:00,F,57,87.0,NaN,NaN,NaN,24.0,98.0,98.6,NaN,NaN,NaN,0,11,2145-11-04 17:39:00


### Aqui apenas estamos organizando a tabela

In [ ]:
cols = df_janelado.columns.tolist()

# Define a nova ordem: traz janela_index e inicio_janela para antes do charttime
nova_ordem = (
    ['subject_id', 'hadm_id', 'stay_id', 'janela_index', 'inicio_janela', 'charttime'] +
    [col for col in cols if col not in ['subject_id', 'hadm_id', 'stay_id', 'janela_index', 'inicio_janela', 'charttime']]
)

# Reorganiza o DataFrame
df_janelas = df_janelado[nova_ordem]
df_janelas.head()

itemid,subject_id,hadm_id,stay_id,janela_index,inicio_janela,charttime,gender,anchor_age,FC,PAS,PAD,PAM,FR,SpO2,Temperatura (°F),Temperatura (°C),Peso,Altura,sepse
0,10000032,29079034,39553978,0,2180-07-23 12:36:00,2180-07-23 12:36:00,F,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.4,152.0,0
1,10000032,29079034,39553978,0,2180-07-23 12:36:00,2180-07-23 14:00:00,F,52,NaN,NaN,NaN,NaN,NaN,NaN,98.7,NaN,NaN,NaN,0
2,10000032,29079034,39553978,0,2180-07-23 12:36:00,2180-07-23 14:11:00,F,52,NaN,84.0,48.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,0
3,10000032,29079034,39553978,0,2180-07-23 12:36:00,2180-07-23 14:12:00,F,52,91.0,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,0
4,10000032,29079034,39553978,0,2180-07-23 12:36:00,2180-07-23 14:13:00,F,52,NaN,NaN,NaN,NaN,NaN,98.0,NaN,NaN,NaN,NaN,0


In [ ]:
mimic = df_janelas.copy()

#Adicionando o rotulo sepse na tabela

Para adicionar o rotulo Sepse tivemos que fazer uma pesquisa na documentação para encontrar onde tem os diagnosticos e foi na tabela

* **diagnoses_icd**

In [ ]:
#Lendo a tabela
diagnosticos = pd.read_csv('/content/diagnoses_icd (1).csv.gz')

In [ ]:
mimic = df

In [ ]:
#Mostrando a tabela
diagnosticos.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9


Após ler a tabela percebemos que precisamos saber qual o codigo ICD da sepse nas versões ICD 9 e 10 que são os que tem no dataset, após pesquisar descobrimos que são os codigos começados em para icd versão 9:
* 038
* 99591
* 99592
---
Para a versão 10 do icd são:
* A41
* R65.2

In [ ]:
# Garante que o código está como string
diagnosticos['icd_code'] = diagnosticos['icd_code'].astype(str)

# Máscara para códigos de sepse (ICD-9)
mask_icd9 = (
    (diagnosticos['icd_version'] == 9) &
    (
        diagnosticos['icd_code'].str.startswith('038') |
        diagnosticos['icd_code'].str.startswith('99591') |
        diagnosticos['icd_code'].str.startswith('99592')
    )
)

# Máscara para códigos de sepse (ICD-10)
mask_icd10 = (
    (diagnosticos['icd_version'] == 10) &
    (
        diagnosticos['icd_code'].str.startswith('A41') |
        diagnosticos['icd_code'].str.startswith('R65.2')
    )
)

# Filtra todos com diagnóstico de sepse
df_sepse = diagnosticos[mask_icd9 | mask_icd10]

# Extrai hadm_ids únicos com sepse
hadm_ids_sepse = df_sepse['hadm_id'].unique()

#ja adicionando como coluna quem tem e quem nao tem sepse
mimic['sepse'] = mimic['hadm_id'].isin(hadm_ids_sepse).astype(int)

In [ ]:
#Mostrando os codigos de SEPSE
df_sepse

,subject_id,hadm_id,seq_num,icd_code,icd_version
128,10000826,21086876,2,99591,9
498,10001401,27012892,2,A4181,10
514,10001401,27060146,2,A419,10
1012,10002013,24848509,1,0389,9
1016,10002013,24848509,5,99592,9
...,...,...,...,...,...
4754790,19997367,20617667,3,03849,9
4754791,19997367,20617667,4,99592,9
4754841,19997367,21508795,1,0380,9
4754858,19997367,21508795,18,99592,9


#Nossa tabela final ficará assim :

In [ ]:
mimic = pd.read_parquet("/content/drive/MyDrive/mimic/mimic.parquet")

In [ ]:
data_dictionary = {
    'subject_id': {
        'Descrição': 'Identificador único do paciente',
        'Tipo': 'Inteiro',
        'Unidade': 'ID'
    },
    'hadm_id': {
        'Descrição': 'Identificador da internação hospitalar',
        'Tipo': 'Inteiro',
        'Unidade': 'ID'
    },
    'stay_id': {
        'Descrição': 'Identificador da permanência em UTI (internação específica)',
        'Tipo': 'Inteiro',
        'Unidade': 'ID'
    },
    'janela_index': {
        'Descrição': 'Índice ordinal da janela de 4 horas, relativa ao primeiro registro',
        'Tipo': 'Inteiro ordinal',
        'Unidade': 'Número da janela (0–9)'
    },
    'inicio_janela': {
        'Descrição': 'Timestamp de início da janela de 4 horas',
        'Tipo': 'Datetime',
        'Unidade': 'YYYY-MM-DD HH:MM:SS'
    },
    'charttime': {
        'Descrição': 'Timestamp da medição específica dentro da janela',
        'Tipo': 'Datetime',
        'Unidade': 'YYYY-MM-DD HH:MM:SS'
    },
    'gender': {
        'Descrição': 'Sexo biológico do paciente',
        'Tipo': 'Categórico',
        'Unidade': 'M (masculino) / F (feminino)'
    },
    'anchor_age': {
        'Descrição': 'Idade aproximada do paciente ao início da internação',
        'Tipo': 'Inteiro',
        'Unidade': 'anos'
    },
    'FC': {
        'Descrição': 'Frequência cardíaca',
        'Tipo': 'Contínuo',
        'Unidade': 'batimentos por minuto (bpm)'
    },
    'PAS': {
        'Descrição': 'Pressão arterial sistólica',
        'Tipo': 'Contínuo',
        'Unidade': 'milímetros de mercúrio (mmHg)'
    },
    'PAD': {
        'Descrição': 'Pressão arterial diastólica',
        'Tipo': 'Contínuo',
        'Unidade': 'milímetros de mercúrio (mmHg)'
    },
    'PAM': {
        'Descrição': 'Pressão arterial média',
        'Tipo': 'Contínuo',
        'Unidade': 'milímetros de mercúrio (mmHg)'
    },
    'FR': {
        'Descrição': 'Frequência respiratória',
        'Tipo': 'Contínuo',
        'Unidade': 'respirações por minuto'
    },
    'SpO2': {
        'Descrição': 'Saturação de oxigênio no sangue',
        'Tipo': 'Contínuo',
        'Unidade': 'porcentagem (%)'
    },
    'Temperatura (°C)': {
        'Descrição': 'Temperatura corporal',
        'Tipo': 'Contínuo',
        'Unidade': 'graus Celsius (°C)'
    },
    'Peso': {
        'Descrição': 'Peso corporal do paciente',
        'Tipo': 'Contínuo',
        'Unidade': 'quilogramas (kg)'
    },
    'Altura': {
        'Descrição': 'Altura do paciente',
        'Tipo': 'Contínuo',
        'Unidade': 'centímetros (cm)'
    },
    'sepse': {
        'Descrição': 'Indicador de sepse (1 = teve sepse, 0 = sem sepse)',
        'Tipo': 'Binário / Categórico',
        'Unidade': '0 ou 1'
    }
}
data_dictionary = pd.DataFrame(data_dictionary).T

Dataset Final:

In [ ]:
mimic.head()

itemid,subject_id,hadm_id,stay_id,charttime,gender,anchor_age,FC,PAS,PAD,PAM,FR,SpO2,Temperatura (°F),Temperatura (°C),Peso,Altura,sepse
0,10000032,29079034,39553978,2180-07-23 12:36:00,F,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.4,152.0,0
1,10000032,29079034,39553978,2180-07-23 14:00:00,F,52,NaN,NaN,NaN,NaN,NaN,NaN,98.7,NaN,NaN,NaN,0
2,10000032,29079034,39553978,2180-07-23 14:11:00,F,52,NaN,84.0,48.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,0
3,10000032,29079034,39553978,2180-07-23 14:12:00,F,52,91.0,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,0
4,10000032,29079034,39553978,2180-07-23 14:13:00,F,52,NaN,NaN,NaN,NaN,NaN,98.0,NaN,NaN,NaN,NaN,0


Esse é o dicionario para entender cada coluna do nosso dataset final

In [ ]:
data_dictionary

,Descrição,Tipo,Unidade
subject_id,Identificador único do paciente,Inteiro,ID
hadm_id,Identificador da internação hospitalar,Inteiro,ID
stay_id,Identificador da permanência em UTI (internaçã...,Inteiro,ID
janela_index,"Índice ordinal da janela de 4 horas, relativa ...",Inteiro ordinal,Número da janela (0–9)
inicio_janela,Timestamp de início da janela de 4 horas,Datetime,YYYY-MM-DD HH:MM:SS
charttime,Timestamp da medição específica dentro da janela,Datetime,YYYY-MM-DD HH:MM:SS
gender,Sexo biológico do paciente,Categórico,M (masculino) / F (feminino)
anchor_age,Idade aproximada do paciente ao início da inte...,Inteiro,anos
FC,Frequência cardíaca,Contínuo,batimentos por minuto (bpm)
PAS,Pressão arterial sistólica,Contínuo,milímetros de mercúrio (mmHg)


Salvando o dataset final para usar nos proximos passos

In [ ]:
mimic.to_csv('/content/drive/MyDrive/mimic.csv', index=False)
mimic.to_parquet('/content/drive/MyDrive/mimic.parquet', index=False)

In [ ]:
mimic.to_parquet('/content/drive/MyDrive/MIMIC_F-C-sem_janelas.parquet', index=False)

# Proximos passos

Agora que temos o dataset mimic pronto podemos passar para a parte de implentar o lstm para isso iremos precisar:

* Normalizar os dados
* transformar de dados tabulares para dados serializados
* Fazer diversos testes para melhorar o resultado do LSTM